In [1]:
import pandas as pd
import datetime 

In [2]:
df = pd.read_csv('../data/parsed_logs_rad_may_2024.csv')

In [3]:
grouped_df = df.groupby(["hostname",  "ecc", "model", "floatthreshold", "microop", "setup_type", "hardenedid", "precision"]).agg({"batchsize": max, "acc_time": sum, "sdc": sum, "critical_sdc": sum})

In [4]:
grouped_df["SDC_rate"] = grouped_df["sdc"] / grouped_df["acc_time"]
grouped_df["Critical_rate"] = grouped_df["critical_sdc"] / grouped_df["acc_time"]

In [5]:
grouped_df.loc[pd.IndexSlice[:, "ON", :]]

batchsize  \
hostname    ecc model                         floatthreshold microop              setup_type     hardenedid precision              
carola20001 ON  swin_base_patch4_window12_384 0.00010        None                 vits           False      fp32              32   
                                                                                                 True       fp32              32   
                                              0.01000        None                 vits           False      fp32              32   
                swin_base_patch4_window7_224  0.00001        None                 vits           False      fp16              32   
                                                                                                            fp32              32   
                                                                                                 True       fp16              32   
                                                                                                            fp32              32   
                vit_base_patch16_384          0.00001        None                 vits           False      fp16              32   
                                                                                                            fp32              32   
                                                                                                 True       fp16              32   
                                                                                                            fp32              32   
carola20002 ON  groundingdino_swinb_cogcoor   0.00001        Attention            grounding_dino False      fp32               1   
                                                                                                 True       fp32               1   
                groundingdino_swint_ogc       0.00001        Attention            grounding_dino False      fp32               1   
                                                                                                 True       fp32               1   
carola20003 ON  swin_base_patch4_window7_224  0.00001        SwinTransformerBlock microbenchmark False      fp16              32   
                                                                                                            fp32              32   

                                                                                                                           acc_time  \
hostname    ecc model                         floatthreshold microop              setup_type     hardenedid precision                 
carola20001 ON  swin_base_patch4_window12_384 0.00010        None                 vits           False      fp32        2383.258363   
                                                                                                 True       fp32         589.985342   
                                              0.01000        None                 vits           False      fp32         115.231299   
                swin_base_patch4_window7_224  0.00001        None                 vits           False      fp16        5764.940648   
                                                                                                            fp32        6617.565160   
                                                                                                 True       fp16        6022.334731   
                                                                                                            fp32        1554.594530   
                vit_base_patch16_384          0.00001        None                 vits           False      fp16        4365.247321   
                                                                                                            fp32        5077.335994   
                                                                                                 True       fp16        2222.

In [6]:
ft_df = df[pd.to_datetime(df["start_dt"]) > datetime.datetime(year=2024, month=5, day=24, hour=12)]
new_group = ft_df.groupby(["hostname",  "ecc", "model", "floatthreshold", "microop", "setup_type", "hardenedid", "precision"]).agg({"batchsize": max, "acc_time": sum, "sdc": sum, "critical_sdc": sum})

In [7]:
new_group["SDC_rate"] = new_group["sdc"] / new_group["acc_time"]
new_group["Critical_rate"] = new_group["critical_sdc"] / new_group["acc_time"]

In [8]:
new_group

batchsize  \
hostname    ecc model                        floatthreshold microop              setup_type     hardenedid precision              
carola20001 ON  swin_base_patch4_window7_224 0.00001        None                 vits           False      fp32              32   
                                                                                                True       fp32              32   
                vit_base_patch16_384         0.00001        None                 vits           False      fp16              32   
                                                                                                           fp32              32   
                                                                                                True       fp16              32   
                                                                                                           fp32              32   
carola20002 ON  groundingdino_swinb_cogcoor  0.00001        Attention            grounding_dino False      fp32               1   
                                                                                                True       fp32               1   
                groundingdino_swint_ogc      0.00001        Attention            grounding_dino False      fp32               1   
                                                                                                True       fp32               1   
carola20003 ON  swin_base_patch4_window7_224 0.00001        SwinTransformerBlock microbenchmark False      fp16              32   
                                                                                                           fp32              32   
carolp22003 OFF SNN_NMNIST                   0.00000        0                    0              0          0                  0   

                                                                                                                          acc_time  \
hostname    ecc model                        floatthreshold microop              setup_type     hardenedid precision                 
carola20001 ON  swin_base_patch4_window7_224 0.00001        None                 vits           False      fp32        5636.983286   
                                                                                                True       fp32        1191.388981   
                vit_base_patch16_384         0.00001        None                 vits           False      fp16        4365.247321   
                                                                                                           fp32        5077.335994   
                                                                                                True       fp16        2222.712865   
                                                                                                           fp32        4902.874839   
carola20002 ON  groundingdino_swinb_cogcoor  0.00001        Attention            grounding_dino False      fp32       15989.089799   
                                                                                                True       fp32        9561.350326   
                groundingdino_swint_ogc      0.00001        Attention            grounding_dino False      fp32       13799.900063   
                                                                                                True       fp32       11795.139750   
carola20003 ON  swin_base_patch4_window7_224 0.00001        SwinTransformerBlock microbenchmark False      fp16        8521.578693   
                                                                                                           fp32        7204.685259   
carolp22003 OFF SNN_NMNIST                   0.00000        0                    0              0          0          16948.446635   

                                                                                                                       sdc  \
hostname    ecc

In [9]:
grouped_df.loc[pd.IndexSlice[:, "OFF", :]]


batchsize  \
hostname    ecc model                         floatthreshold microop   setup_type     hardenedid precision              
carola20001 OFF swin_base_patch4_window12_384 0.01           None      vits           False      fp32               8   
                vit_base_patch16_224          0.01           None      vits           False      fp32               8   
                                                                                      True       fp32               8   
                                              0.10           None      vits           False      fp16               8   
                                                                                      True       fp16               8   
                vit_base_patch16_384          0.01           Attention microbenchmark False      fp32               8   
                                                             Block     microbenchmark False      fp32               8   
carolp20002 OFF groundingdino_swinb_cogcoor   0.01           Attention grounding_dino False      fp32               1   
                                                                                      True       fp32               1   
                                              0.10           Attention grounding_dino False      fp32               1   
                                                                                      True       fp32               1   
                groundingdino_swint_ogc       0.01           Attention grounding_dino False      fp32               1   
                                                                                      True       fp32               1   
carolp22003 OFF SNN_NMNIST                    0.00           0         0              0          0                  0   

                                                                                                                acc_time  \
hostname    ecc model                         floatthreshold microop   setup_type     hardenedid precision                 
carola20001 OFF swin_base_patch4_window12_384 0.01           None      vits           False      fp32         617.650461   
                vit_base_patch16_224          0.01           None      vits           False      fp32        1899.688553   
                                                                                      True       fp32        1463.919021   
                                              0.10           None      vits           False      fp16        1669.635372   
                                                                                      True       fp16         196.248820   
                vit_base_patch16_384          0.01           Attention microbenchmark False      fp32        1195.039556   
                                                             Block     microbenchmark False      fp32        2698.228651   
carolp20002 OFF groundingdino_swinb_cogcoor   0.01           Attention grounding_dino False      fp32        3923.360195   
                                                                                      True       fp32        1427.972256   
                                              0.10           Attention grounding_dino False      fp32        5554.948415   
                                                                                      True       fp32         642.790403   
                groundingdino_swint_ogc       0.01           Attention grounding_dino False      fp32        9353.682741   
                                                                                      True       fp32        3337.683801   
carolp22003 OFF SNN_NMNIST                    0.00           0         0              0          0          29347.679238   

                                                                                                             sdc  \
hostname    ecc model                         floatthreshold microop  